# Modelos usando GPU: RAPIDs

Mediante WSL podemos trabajar con la versión de Linux de RAPIDs. Con ella, hemos entrenado varios modelos.

In [1]:
import cudf
import cuml
from cupy import asnumpy
from joblib import dump, load
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pickle
import pyarrow.parquet as pq

# https://docs.rapids.ai/api/cuml/stable/api/
from cuml.ensemble import RandomForestRegressor as cuRF
from cuml.linear_model import LinearRegression as cuLR
from cuml.linear_model import ElasticNet
from cuml.svm import SVR
from cuml.metrics import mean_squared_error, r2_score

import random
random.seed(33)

def recursive_loops(arrays, results, depth=0, current=[]):
    if depth == len(arrays):
        results.append(current)
        return
    for item in arrays[depth]:
        recursive_loops(arrays, results, depth + 1, current + [item])

In [2]:
input_path = "/mnt/c/Users/domin/Desktop/TFM/output/"
import pandas as pd
import pickle

In [3]:
train_df_pd = pd.read_pickle(input_path + "train_df_pipeline.pkl")
test_df_pd = pd.read_pickle(input_path + "test_df_pipeline.pkl")

# train_df_pd = pd.read_pickle(input_path + "train_df.pkl")
# test_df_pd = pd.read_pickle(input_path + "test_df.pkl")

train_df_pd.head()

,Price,Date.of.Transfer,Property.Type,Old.New,Duration,Town.City,District,County,org_property_type,org_duration,...,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_q1,log_q3,log_mean_price,log_year_to_year_change,org_mean_price,org_price_boxcox
0,300000,1995-01-01,3,Nuevo,1,london,merton,greater london,Piso/Apartamento,Alquiler,...,-0.057229,1.687222,-1.338552,3.112323,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,14.442841
1,70500,1995-01-01,1,Segunda_mano,0,horsham,horsham,west sussex,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.996887,1.007904,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,12.583137
2,35000,1995-01-01,0,Segunda_mano,0,dewsbury,kirklees,west yorkshire,Adosado,Propiedad,...,-0.057229,0.848829,0.928138,0.003143,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.703994
3,41000,1995-01-01,1,Segunda_mano,0,gloucester,gloucester,gloucestershire,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.072118,0.228881,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.901504
4,125000,1995-01-01,2,Segunda_mano,0,macclesfield,macclesfield,cheshire,Unifamiliar,Propiedad,...,-0.057229,-0.827957,0.698371,1.837439,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,13.311820


In [4]:
# for var in test_df_pd.columns[test_df_pd.isnull().sum()>0]:
#     test_df_pd.loc[test_df_pd[var].isnull(), var] = np.mean(test_df_pd[var])
test_df_pd.isnull().sum().sum()

0

In [5]:
# Convert pandas DataFrame to cudf DataFrame for GPU processing
train_df = cudf.DataFrame.from_pandas(train_df_pd)
test_df = cudf.DataFrame.from_pandas(test_df_pd)

# Check the loaded data
del train_df_pd, test_df_pd
display(train_df.head())
display(test_df.head())

,Price,Date.of.Transfer,Property.Type,Old.New,Duration,Town.City,District,County,org_property_type,org_duration,...,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_q1,log_q3,log_mean_price,log_year_to_year_change,org_mean_price,org_price_boxcox
0,300000,1995-01-01,3,Nuevo,1,london,merton,greater london,Piso/Apartamento,Alquiler,...,-0.057229,1.687222,-1.338552,3.112323,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,14.442841
1,70500,1995-01-01,1,Segunda_mano,0,horsham,horsham,west sussex,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.996887,1.007904,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,12.583137
2,35000,1995-01-01,0,Segunda_mano,0,dewsbury,kirklees,west yorkshire,Adosado,Propiedad,...,-0.057229,0.848829,0.928138,0.003143,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.703994
3,41000,1995-01-01,1,Segunda_mano,0,gloucester,gloucester,gloucestershire,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.072118,0.228881,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.901504
4,125000,1995-01-01,2,Segunda_mano,0,macclesfield,macclesfield,cheshire,Unifamiliar,Propiedad,...,-0.057229,-0.827957,0.698371,1.837439,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,13.311820


,Price,Date.of.Transfer,Property.Type,Old.New,Duration,Town.City,District,County,org_property_type,org_duration,...,job_density,density_population_hectare,is_coastal,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_year_to_year_change,org_mean_price,org_price_boxcox
0,234000,2021-01-01,2,Segunda_mano,1,liverpool,knowsley,merseyside,Unifamiliar,Alquiler,...,-0.051706,-0.122306,-0.603309,-0.057229,1.128294,1.086680,-0.398926,1.918864,273204.058599,14.119733
1,255000,2021-01-01,3,Segunda_mano,1,st ives,cornwall,cornwall,Piso/Apartamento,Alquiler,...,-0.029515,-0.741602,1.657527,-0.057229,-0.618358,2.215009,0.092717,1.918864,273204.058599,14.231305
2,450000,2021-01-01,0,Segunda_mano,0,harrow,harrow,greater london,Adosado,Propiedad,...,-0.201499,1.104223,-0.603309,-0.057229,1.687222,-1.302514,1.046355,1.918864,273204.058599,14.973763
3,360000,2021-01-01,3,Segunda_mano,1,west wickham,bromley,greater london,Piso/Apartamento,Alquiler,...,-0.134924,0.026486,-0.603309,-0.057229,1.687222,-1.329589,1.075334,1.918864,273204.058599,14.681015
4,590000,2021-01-01,1,Segunda_mano,0,orpington,bromley,greater london,Semi-adosado,Propiedad,...,-0.134924,0.026486,-0.603309,-0.057229,-0.827957,-1.297941,1.075334,1.918864,273204.058599,15.330982


In [6]:
!nvidia-smi

Sun Nov  3 18:09:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P3              8W /   80W |    1387MiB /   6144MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Modelos

In [7]:
import pandas as pd
import pyarrow.parquet
# https://stackoverflow.com/questions/41567081/get-schema-of-parquet-file-in-python

def read_parquet_schema_df(uri: str) -> pd.DataFrame:
    """Return a Pandas dataframe corresponding to the schema of a local URI of a parquet file.

    The returned dataframe has the columns: column, pa_dtype
    """
    # Ref: https://stackoverflow.com/a/64288036/
    schema = pyarrow.parquet.read_schema(uri, memory_map=True)
    schema = pd.DataFrame(({"column": name, "pa_dtype": str(pa_dtype)} for name, pa_dtype in zip(schema.names, schema.types)))
    schema = schema.reindex(columns=["column", "pa_dtype"], fill_value=pd.NA)  # Ensures columns in case the parquet file has an empty dataframe.
    return schema

train_schema = read_parquet_schema_df('../../data/parquet/train/train.parquet')

all_numeric_vars = train_schema.loc[train_schema["pa_dtype"].isin(["timestamp[us]", "string"]) == False, "column"][:-1].values # last is an index
independent_variables = [v for v in all_numeric_vars if v not in ["Price_boxcox", "Price", "org_price_boxcox", "org_mean_price"]]
target_variable = "Price_boxcox"

### Selección de variables mediante Feature Importance

In [8]:
X_train = train_df.loc[:, independent_variables].astype("float32")
y_train = train_df["Price_boxcox"].astype("float32")

# X_test = test_df.loc[:, independent_variables].astype("float32")
# y_test= test_df["Price_boxcox"].astype("float32")

Creamos un modelo Random Forest arbitrario para evaluar el aumento en el error cuando se cambian las variables:

In [9]:
%%time
model = cuRF(n_estimators=100, max_depth=10, random_state=33,
         n_streams=1, bootstrap=True)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_train, y_pred)
print(f'R² Score: {r2}')

Mean Squared Error: 0.03595009073615074
R² Score: 0.9641258716583252
CPU times: user 26.4 s, sys: 13.9 s, total: 40.3 s
Wall time: 38.8 s


In [10]:
# Step 1: Calculate the baseline error on the original data
baseline_error = mse

# Step 2: Initialize a dictionary to store importance scores
feature_importances = {}

# Step 3: Calculate importance for each feature
for col in X_train.columns:
    # Make a copy of X to permute this feature
    X_permuted = X_train.copy()
    
    # Step 4: Permute the column values (shuffling breaks the association with the target)
    X_permuted[col] = X_permuted[col].sample(frac=1).reset_index(drop=True)
    
    # Step 5: Predict with the permuted data and calculate the new error
    permuted_predictions = model.predict(X_permuted)
    permuted_error = mean_squared_error(y_train, permuted_predictions)
    
    # Step 6: Calculate importance as the increase in error from the baseline
    feature_importance = permuted_error - baseline_error
    feature_importances[col] = feature_importance

# Convert to DataFrame for sorting and readability
feature_importance_df = cudf.DataFrame(list(feature_importances.items()), columns=['Feature', 'Importance'])
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feature_importance_df)


                       Feature    Importance
12        price_compare_london  2.924565e-01
31    log_price_compare_london  1.945822e-01
3                         Year  1.099439e-01
8               is_top_outlier  6.003292e-02
7                           q3  3.498458e-02
32                      log_q1  2.547155e-02
6                           q1  2.357341e-02
10                  mean_price  1.718652e-02
33                      log_q3  1.199350e-02
34              log_mean_price  8.947540e-03
30          distance_to_london  7.655680e-03
1                     Duration  1.510356e-03
29             spatial_cluster  1.453992e-03
26  density_population_hectare  1.334012e-03
11         year_to_year_change  1.327071e-03
35     log_year_to_year_change  1.297381e-03
21                       BandD  1.161467e-03
19                       BandB  6.281435e-04
25                 job_density  4.785508e-04
20                       BandC  2.768040e-04
23                       BandF  2.330840e-04
2         

In [11]:
feature_importance_df.to_csv("../../output/R/feature_importance.csv")

Seleccionamos las variables (nos basamos en el gráfico generado en R, concretamente, descartamos las variables que tienen importancia prácticamente cero):

![feature importance](feature_importance.jpeg)

In [12]:
independent_variables = [v for v in independent_variables if v not in 
                         (
                             # 'log_year_to_year_change', #
                             # 'Duration', #
                             'BandG',
                             'is_coastal',
                             'BandF',
                             'total_people',
                             'total_houses',
                             'people_working',
                             'people_studying',
                             # 'spatial_cluster',
                             'day',
                             'is_bottom_outlier',
                             'is_isle',
                             'fixed_effect',
                             # 'year_to_year_change', #
                             'month', #
                             'BandA',
                             'job_density',
                             'First_hand', #
                             'BandC',
                             # 'density_population_hectare',
                             'BandB',
                             'Property.Type', #
                             'BandE',
                             # 'BandD'
                         )]

In [13]:
X_train = train_df.loc[:, independent_variables].astype("float32")
y_train = train_df["Price_boxcox"].astype("float32")

X_test = test_df.loc[:, independent_variables].astype("float32")
y_test= test_df["Price_boxcox"].astype("float32")

In [14]:
del train_df, test_df

In [15]:
!nvidia-smi

Sun Nov  3 18:10:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P3             12W /   80W |    2313MiB /   6144MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Random Forest Regressor
Es necesario mantener n_streams a 1, porque se asegura el efecto determinista de fijar la semilla aleatoria.

In [16]:
%%time
rf = cuRF(n_estimators=100, max_depth=10, random_state=33,
         n_streams=1, bootstrap=True)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

Mean Squared Error: 0.5682413578033447
R² Score: 0.23278355598449707
CPU times: user 21.4 s, sys: 11.4 s, total: 32.8 s
Wall time: 32.1 s


In [17]:
del rf

#### Grid Search: Random Forest

In [18]:
# Make grid
parameters = [
    [100, 200, 300], # n_estimators
    [3, 5, 7, 10], # max_depth
]

results = []
recursive_loops(arrays=parameters, results=results)

resultados_rf = pd.DataFrame(results)
resultados_rf.columns = ["n_estimators", "max_depth"]
resultados_rf["mse"] = np.float32(0)
resultados_rf["r2"] = np.float32(0)

resultados_rf.shape

(12, 4)

In [19]:
nrows = resultados_rf.shape[0]
for idx in np.arange(nrows):
    print(f"\r{idx+1}/{nrows}", end="")
    kwargs = {
        "n_estimators": resultados_rf.loc[idx, "n_estimators"],
        "max_depth": resultados_rf.loc[idx, "max_depth"],
        "random_state": 33,
        "n_streams": 1,
        "bootstrap": True, 
    }

    model = cuRF(**kwargs)
    sample_size = 100000 * 2
    model.fit(X_train.sample(n=sample_size, random_state=33), 
              y_train.sample(n=sample_size, random_state=33))
    
    # Evaluate model
    y_pred = model.predict(X_test)

    resultados_rf.loc[idx, "mse"] = mean_squared_error(y_test, y_pred)
    resultados_rf.loc[idx, "r2"] = r2_score(y_test, y_pred)

12/12

In [20]:
del model
resultados_rf = resultados_rf.sort_values(by="r2", ascending=False).reset_index().drop("index", axis=1)
resultados_rf.head()

,n_estimators,max_depth,mse,r2
0,100,5,0.393211,0.469102
1,300,5,0.398051,0.462567
2,200,5,0.400537,0.459212
3,300,3,0.443007,0.401870
4,100,3,0.445286,0.398793


### Linear Regressor

In [21]:
%%time
# https://docs.rapids.ai/api/cuml/stable/estimator_intro/#Linear-regression-and-R^2-score
model = cuLR(fit_intercept = True,
                      normalize = True,
                      algorithm = 'eig')

model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

/home/domingo/miniforge3/envs/rapids-24.08/lib/python3.11/site-packages/cuml/internals/api_decorators.py:382: UserWarning: Starting from version 23.08, the new 'copy_X' parameter defaults to 'True', ensuring a copy of X is created after passing it to fit(), preventing any changes to the input, but with increased memory usage. This represents a change in behavior from previous versions. With `copy_X=False` a copy might still be created if necessary. Explicitly set 'copy_X' to either True or False to suppress this warning.
  return init_func(self, *args, **filtered_kwargs)


Mean Squared Error: 1.2426419258117676
R² Score: -0.6777646541595459
CPU times: user 88.5 ms, sys: 106 ms, total: 195 ms
Wall time: 260 ms


In [22]:
del model

#### Grid Search: Linear Regressor

In [23]:
# Make grid
parameters = [
    [True, False], # fit_intercept
    [True, False], # normalize
    ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi'] # algorithm
]

results = []
recursive_loops(arrays=parameters, results=results)

resultados_lm = pd.DataFrame(results)
resultados_lm.columns = ["fit_intercept", "normalize", "algorithm"]
resultados_lm["mse"] = np.float32(0)
resultados_lm["r2"] = np.float32(0)

resultados_lm.shape

(20, 5)

In [24]:
nrows = resultados_lm.shape[0]
for idx in np.arange(nrows):
    print(f"\r{idx+1}/{nrows}", end="")
    kwargs = {
        "fit_intercept": resultados_lm.loc[idx, "fit_intercept"],
        "normalize": resultados_lm.loc[idx, "normalize"],
        "algorithm": resultados_lm.loc[idx, "algorithm"],
        "copy_X": True,
    }

    model = cuLR(**kwargs)
    model.fit(X_train, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test)

    resultados_lm.loc[idx, "mse"] = mean_squared_error(y_test, y_pred)
    resultados_lm.loc[idx, "r2"] = r2_score(y_test, y_pred)

20/20

In [25]:
del model
resultados_lm = resultados_lm.sort_values(by="r2", ascending=False).reset_index().drop("index", axis=1)
resultados_lm.head()

,fit_intercept,normalize,algorithm,mse,r2
0,True,True,eig,1.242642,-0.677765
1,True,True,qr,1.279373,-0.727357
2,True,True,svd,1.279373,-0.727358
3,False,True,qr,1.468717,-0.983002
4,False,True,svd,1.468718,-0.983003


### SVM (Support Vector Machine)

No da buenos resultados y tiene un alto coste computacional.

In [26]:
# %%time
# # https://docs.rapids.ai/api/cuml/stable/estimator_intro/#Linear-regression-and-R^2-score
# model = SVR(kernel='rbf', gamma='scale', C=10, epsilon=0.1)
# model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = model.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse}')

# r2 = r2_score(y_test, y_pred)
# print(f'R² Score: {r2}')

In [27]:
# del model

### Elastic Net

In [110]:
%%time
# https://docs.rapids.ai/api/cuml/stable/estimator_intro/#Linear-regression-and-R^2-score
model = ElasticNet(alpha = 0.1, l1_ratio=0.5, solver='qn')
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

Mean Squared Error: 0.4925053119659424
R² Score: 0.33503925800323486
CPU times: user 730 ms, sys: 28.7 ms, total: 759 ms
Wall time: 763 ms


In [111]:
del model

#### Grid Search: Elastic Net

In [127]:
# Make grid
parameters = [
    [0.1, 0.5, 0.7, 1], # alpha
    [0.3, 0.5, 0.7, 1], # l1_ratio
    [1e-3, 1e-4, 1e-2] # tol
]

results = []
recursive_loops(arrays=parameters, results=results)

resultados_en = pd.DataFrame(results)
resultados_en.columns = ["alpha", "l1_ratio", "tol"]
resultados_en["mse"] = np.float32(0)
resultados_en["r2"] = np.float32(0)

resultados_en.shape

(48, 5)

In [128]:
nrows = resultados_en.shape[0]
for idx in np.arange(nrows):
    print(f"\r{idx+1}/{nrows}", end="")
    kwargs = {
        "alpha": resultados_en.loc[idx, "alpha"],
        "l1_ratio": resultados_en.loc[idx, "l1_ratio"],
        "tol": resultados_en.loc[idx, "tol"],
    }

    model = ElasticNet(**kwargs)
    model.fit(X_train, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test)

    resultados_en.loc[idx, "mse"] = mean_squared_error(y_test, y_pred)
    resultados_en.loc[idx, "r2"] = r2_score(y_test, y_pred)

48/48

In [129]:
del model
resultados_en = resultados_en.sort_values(by="r2", ascending=False).reset_index().drop("index", axis=1)
resultados_en.head()

,alpha,l1_ratio,tol,mse,r2
0,1.0,0.5,0.0100,0.708429,0.043508
1,1.0,0.5,0.0001,0.708429,0.043508
2,1.0,0.5,0.0010,0.708429,0.043508
3,0.7,1.0,0.0100,0.715108,0.034490
4,0.7,1.0,0.0010,0.715154,0.034428


### XGBoost Regressor

In [21]:
# https://www.kaggle.com/code/titericz/rapids-xgboost#XGBoost-GPU
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=33)

dtrain = xgb.DMatrix(X_train2, y_train2)
dvalid = xgb.DMatrix(X_val, y_val)
dtest = xgb.DMatrix(X_test, y_test)

del X_train2, y_train2, X_val, y_val, X_test

In [22]:
# XGBoost
params = {
    "subsample": 0.5,
    "colsample_bytree": 0.1,
    "max_depth": 3,
    "learning_rate": 0.05,
    "eval_metric": "rmse",
    "nthread": -1,
    "device": "cuda",
    "max_bin": 255, 
    'seed' : 33,
}

model = xgb.train(
    params,
    dtrain,
    num_boost_round = 2000,
    evals = [(dtrain, "train"), (dvalid, "valid")],
    verbose_eval = 100,
    early_stopping_rounds = 50,
    # feval=evalerror
)

# y_train[valid_idx] = model.predict(dvalid)
y_pred = model.predict(dtest)

[0]	train-rmse:0.98437	valid-rmse:0.98369
[100]	train-rmse:0.48307	valid-rmse:0.48333
[200]	train-rmse:0.36064	valid-rmse:0.36111
[300]	train-rmse:0.28050	valid-rmse:0.28107
[400]	train-rmse:0.23820	valid-rmse:0.23868
[500]	train-rmse:0.21379	valid-rmse:0.21422
[600]	train-rmse:0.19821	valid-rmse:0.19859
[700]	train-rmse:0.18857	valid-rmse:0.18894
[800]	train-rmse:0.17694	valid-rmse:0.17728
[900]	train-rmse:0.16990	valid-rmse:0.17023
[1000]	train-rmse:0.16262	valid-rmse:0.16292
[1100]	train-rmse:0.15643	valid-rmse:0.15671
[1200]	train-rmse:0.15111	valid-rmse:0.15138
[1300]	train-rmse:0.14704	valid-rmse:0.14730
[1400]	train-rmse:0.14419	valid-rmse:0.14444
[1500]	train-rmse:0.13996	valid-rmse:0.14020
[1600]	train-rmse:0.13619	valid-rmse:0.13642
[1700]	train-rmse:0.13428	valid-rmse:0.13453
[1800]	train-rmse:0.13158	valid-rmse:0.13183
[1900]	train-rmse:0.12875	valid-rmse:0.12899
[1999]	train-rmse:0.12727	valid-rmse:0.12751


In [23]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

Mean Squared Error: 0.6246811151504517
R² Score: 0.15658092498779297


In [24]:
del model

#### Grid Search: XGBoost
Vamos a hacer una versión personalizada para evitar exceder el límite de memoria de la GPU.

In [25]:


# Make grid
# parameters = [
#     np.arange(0.1, 1, 0.4), # subsample
#     np.arange(0.01, 0.2, 0.05), # colsample_bytree
#     np.arange(1, 10, 3), # maxdepth
#     np.array([0.01, 0.05, 0.2]) # learning_rate
# ]

parameters = [
    [0.5, 0.25], # subsample
    [0.1], # colsample_bytree
    [3, 5, 8], # maxdepth
    [0.01] # learning_rate
]

results = []
recursive_loops(arrays=parameters, results=results)
resultados_xgboost = pd.DataFrame(results)
resultados_xgboost.columns = ["subsample", "colsample_bytree", 
                              "maxdepth", "learning_rate"]
resultados_xgboost["mse"] = np.float32(0)
resultados_xgboost["r2"] = np.float32(0)
resultados_xgboost.shape

(6, 6)

In [26]:
nrows = resultados_xgboost.shape[0]
for idx in np.arange(nrows):
    print(f"\r{idx+1}/{nrows}", end="")
    params = {
        "subsample": resultados_xgboost.loc[idx, "subsample"],
        "colsample_bytree": resultados_xgboost.loc[idx, "colsample_bytree"],
        "max_depth": resultados_xgboost.loc[idx, "maxdepth"],
        "learning_rate": resultados_xgboost.loc[idx, "learning_rate"],
        "eval_metric": "rmse",
        "device": "cuda",
        "max_bin": 255, 
        'seed' : 33
    }

    model = xgb.train(
    params,
    dtrain,
    num_boost_round = 1000,
    evals = [(dtrain, "train"), (dvalid, "valid")],
    verbose_eval = 0,
    early_stopping_rounds = 50
    )

    # Evaluate model
    y_pred = model.predict(dtest)

    resultados_xgboost.loc[idx, "mse"] = mean_squared_error(y_test, y_pred)
    # print(f'Mean Squared Error: {mse}')
    
    resultados_xgboost.loc[idx, "r2"] = r2_score(y_test, y_pred)
    # print(f'R² Score: {r2}')

6/6

In [27]:
del model
resultados_xgboost = resultados_xgboost.sort_values(by="r2", ascending=False).reset_index().drop("index", axis=1)
resultados_xgboost.head()

,subsample,colsample_bytree,maxdepth,learning_rate,mse,r2
0,0.25,0.1,8,0.01,0.378538,0.488913
1,0.50,0.1,8,0.01,0.378619,0.488804
2,0.50,0.1,3,0.01,0.383088,0.482770
3,0.50,0.1,5,0.01,0.383344,0.482424
4,0.25,0.1,5,0.01,0.383405,0.482342


## Modelo final

In [28]:
# XGBoost
results = resultados_xgboost.iloc[0,:]
params = {
    "subsample": results["subsample"],
    "colsample_bytree": results["colsample_bytree"],
    "max_depth": np.int32(results["maxdepth"]),
    "learning_rate": results["learning_rate"],
    "eval_metric": "rmse",
    "device": "cuda",
    "max_bin": 255, 
    'seed' : 33,
}

model = xgb.train(
    params,
    dtrain,
    num_boost_round = 2000,
    evals = [(dtrain, "train"), (dvalid, "valid")],
    verbose_eval = 100,
    early_stopping_rounds = 50,
    # feval=evalerror
)

# y_train[valid_idx] = model.predict(dvalid)
y_pred = model.predict(dtest)


[0]	train-rmse:0.99773	valid-rmse:0.99705
[100]	train-rmse:0.79668	valid-rmse:0.79630
[200]	train-rmse:0.67120	valid-rmse:0.67104
[300]	train-rmse:0.57427	valid-rmse:0.57437
[400]	train-rmse:0.50402	valid-rmse:0.50430
[500]	train-rmse:0.44799	valid-rmse:0.44840
[600]	train-rmse:0.40294	valid-rmse:0.40343
[700]	train-rmse:0.37010	valid-rmse:0.37065
[800]	train-rmse:0.35827	valid-rmse:0.35880
[900]	train-rmse:0.34437	valid-rmse:0.34491
[1000]	train-rmse:0.32949	valid-rmse:0.33005
[1100]	train-rmse:0.31693	valid-rmse:0.31749
[1200]	train-rmse:0.30200	valid-rmse:0.30258
[1300]	train-rmse:0.28426	valid-rmse:0.28485
[1400]	train-rmse:0.27852	valid-rmse:0.27910
[1500]	train-rmse:0.27043	valid-rmse:0.27100
[1600]	train-rmse:0.26225	valid-rmse:0.26282
[1700]	train-rmse:0.25565	valid-rmse:0.25621
[1800]	train-rmse:0.25048	valid-rmse:0.25104
[1900]	train-rmse:0.24399	valid-rmse:0.24454
[1999]	train-rmse:0.24088	valid-rmse:0.24142


In [29]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

Mean Squared Error: 0.35234957933425903
R² Score: 0.5242719054222107


In [30]:
del dtrain, dvalid, dtest, y_pred, y_test
# del model

In [31]:
import pickle
with open("../../output/xgboost_final.pkl", "wb") as file: # file is a variable for storing the newly created file, it can be anything.
    pickle.dump(model, file) # Dump function is used to write the object into the created file in byte format.


### Evaluar todo test

No merece la pena, sale igual que con la muestra pequeña.

In [32]:
import pandas as pd
import pyarrow.parquet
# https://stackoverflow.com/questions/41567081/get-schema-of-parquet-file-in-python

def read_parquet_schema_df(uri: str) -> pd.DataFrame:
    """Return a Pandas dataframe corresponding to the schema of a local URI of a parquet file.

    The returned dataframe has the columns: column, pa_dtype
    """
    # Ref: https://stackoverflow.com/a/64288036/
    schema = pyarrow.parquet.read_schema(uri, memory_map=True)
    schema = pd.DataFrame(({"column": name, "pa_dtype": str(pa_dtype)} for name, pa_dtype in zip(schema.names, schema.types)))
    schema = schema.reindex(columns=["column", "pa_dtype"], fill_value=pd.NA)  # Ensures columns in case the parquet file has an empty dataframe.
    return schema

train_schema = read_parquet_schema_df('../../data/parquet/train/train.parquet')
train_schema.head()

,column,pa_dtype
0,Price,int64
1,Date.of.Transfer,timestamp[us]
2,Property.Type,int64
3,Old.New,string
4,Duration,int64


In [33]:
# all_numeric_vars = train_schema.loc[train_schema["pa_dtype"].isin(["timestamp[us]", "string"]) == False, "column"][:-1].values # last is an index
# independent_variables = [v for v in all_numeric_vars if v not in ["Price_boxcox", "Price", "org_price_boxcox",
#                                                                  "org_mean_price", "year_to_year_change"]]
# target_variable = "Price_boxcox"

In [34]:
import random
random.seed(33)

# # Train: random sample
# parquet_file = pq.ParquetFile('../../data/parquet/train/train.parquet')
# n_batches_train = np.array([1 for _ in parquet_file.iter_batches()]).sum()

# indeces = list(range(n_batches_train))[1:] # First must be 0, even if it adds one more than expected
# # n = 30
# n = int(0.02 * len(indeces))
# chosen_batches_train = [0] + random.sample(indeces, n)

# Test and validation partitions
parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
n_batches_test = np.array([1 for _ in parquet_file.iter_batches()]).sum()

indeces = list(range(n_batches_test))
# n = int(0.25 * len(indeces))
# n = 1
chosen_batches_validation = 0
chosen_batches_test = [idx for idx in indeces if idx not in [chosen_batches_validation]]

def make_dDF(batch):
    batch_df = cudf.DataFrame.from_arrow(pyarrow.Table.from_batches([batch]))
    X = batch_df.loc[:, independent_variables].astype("float32")
    y = batch_df["Price_boxcox"].astype("float32")
    return xgb.DMatrix(X, y)

In [35]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks


results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    if idx not in chosen_batches_test:
        continue
    
    batch_df = batch.to_pandas()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)


Batch 32 of 32

In [36]:
results_df = results_df.astype("float32")
results_df.mean()

R2     0.520859
MSE    0.349588
dtype: float32

### Evaluación del modelo final en zonas más generales

#### Mensual

In [39]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "District", "Town.City"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [40]:
results_df.astype("float32").mean()

R2     0.556684
MSE    0.202884
dtype: float32

In [37]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "District"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)


Batch 32 of 32

In [38]:
results_df.astype("float32").mean()

R2     0.822729
MSE    0.057350
dtype: float32

In [41]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "County"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [42]:
results_df.astype("float32").mean()

R2     0.703407
MSE    0.068884
dtype: float32

#### Anual

In [45]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "District", "Town.City"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [46]:
results_df.astype("float32").mean()

R2     0.563547
MSE    0.193746
dtype: float32

In [43]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "District"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)


Batch 32 of 32

In [44]:
results_df.astype("float32").mean()

R2     0.852348
MSE    0.046635
dtype: float32

In [47]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "County"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [48]:
results_df.astype("float32").mean()

R2     0.736331
MSE    0.059756
dtype: float32

# Diario

In [51]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "day", "District", "Town.City"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [52]:
results_df.astype("float32").mean()

R2     0.537911
MSE    0.268691
dtype: float32

In [49]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "day", "District"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)


Batch 32 of 32

In [50]:
results_df.astype("float32").mean()

R2     0.671513
MSE    0.146793
dtype: float32

In [53]:
import pyarrow.parquet as pq
# https://stackoverflow.com/questions/59098785/is-it-possible-to-read-parquet-files-in-chunks

results_df = pd.DataFrame(columns=["R2", "MSE"], index=np.arange(n_batches_test))

parquet_file = pq.ParquetFile('../../data/parquet/test/test.parquet')
for idx, batch in enumerate(parquet_file.iter_batches()):
    print(f"\rBatch {idx} of {n_batches_test-1}", end="")

    # if idx not in chosen_batches_test:
    #     continue
    
    batch_df = batch.to_pandas().\
        groupby(["Year", "month", "day", "County"]).mean(numeric_only=True).\
            reset_index()
    
    X_test = batch_df.loc[:, independent_variables].astype("float32")
    y_test = batch_df["Price_boxcox"].astype("float32")
    dtest = xgb.DMatrix(X_test, y_test)
    del X_test
    
    y_pred = model.predict(dtest)

    results_df.iloc[idx, 1] = mean_squared_error(y_test, y_pred)
    results_df.iloc[idx, 0] = r2_score(y_test, y_pred)

Batch 32 of 32

In [54]:
results_df.astype("float32").mean()

R2     0.565656
MSE    0.141738
dtype: float32